**_Note: This notebook contains ALL the code for Sections 12.7 through 12.11 because all the snippets in these sections are consecutively numbered in the text._**

# 12.7 Authenticating with Twitter Via Tweepy to Access Twitter v2 APIs

In [ ]:
import tweepy

In [ ]:
import keys

### Creating a Client Object

In [ ]:
client = tweepy.Client(bearer_token=keys.bearer_token,
                       wait_on_rate_limit=True)

# 12.8 Getting Information About a Twitter Account

In [ ]:
nasa = client.get_user(username='NASA', 
    user_fields=['description', 'public_metrics'])

### tweepy.Response Object

### Getting a User’s Basic Account Information

In [ ]:
nasa.data.id

In [ ]:
nasa.data.name

In [ ]:
nasa.data.username

In [ ]:
nasa.data.description

### Getting the Number of Accounts That Follow This Account and the Number of Accounts This Account Follows

In [ ]:
nasa.data.public_metrics['followers_count']

In [ ]:
nasa.data.public_metrics['following_count']

### Getting Your Own Account’s Information

# 12.9 Intro to Tweepy `Paginator`s: Getting More than One Page of Results

# 12.9.1 Determining an Account’s Followers 

In [ ]:
followers = []

### Creating a `Paginator`

In [ ]:
paginator = tweepy.Paginator(
    client.get_users_followers, nasa.data.id, max_results=5)

### Getting Results

In [ ]:
for follower in paginator.flatten(limit=10):
    followers.append(follower.username)

In [ ]:
print('Followers:', 
      ' '.join(sorted(followers, key=lambda s: s.lower())))

# 12.9.2 Determining Whom an Account Follows 

In [ ]:
following = []

In [ ]:
paginator = tweepy.Paginator(
    client.get_users_following, nasa.data.id, max_results=5)

In [ ]:
for user_followed in paginator.flatten(limit=10):
    following.append(user_followed.username)

In [ ]:
print('Following:', 
      ' '.join(sorted(following, key=lambda s: s.lower())))

# 12.9.3 Getting a User’s Recent Tweets

In [ ]:
nasa_tweets = client.get_users_tweets(
    id=nasa.data.id, max_results=5)

In [ ]:
for tweet in nasa_tweets.data:
    print(f"NASA: {tweet.data['text']}\n")

### Grabbing Recent Tweets from Your Own Timeline

# 12.10 Searching Recent Tweets; Intro to Twitter v2 API Search Operators

### Utility Function `print_tweets` from `tweetutilities.py`

In [ ]:
from tweetutilities import print_tweets

```python
def print_tweets(tweets):
    # translator to autodetect source language and return English
    translator = GoogleTranslator(source='auto', target='en')

    """For each tweet in tweets, display the username of the sender
    and tweet text. If the language is not English, translate the text 
    with the deep-translator library's GoogleTranslator."""
    for tweet, user in zip(tweets.data, tweets.includes['users']):
        print(f'{user.username}:', end=' ')

        if 'en' in tweet.lang:
            print(f'{tweet.text}\n')
        elif 'und' not in tweet.lang: # translate to English first
            print(f'\n  ORIGINAL: {tweet.text}')
            print(f'TRANSLATED: {translator.translate(tweet.text)}\n')
```

### Searching for Specific Words

In [ ]:
tweets = client.search_recent_tweets(
    query='Webb Space Telescope', 
    expansions=['author_id'], tweet_fields=['lang'])

In [ ]:
print_tweets(tweets)

### Searching with Twitter v2 API Search Operators

### Searching with Twitter v2 API Search Operators

### Operator Documentation and Tutorial

### Searching for Tweets From NASA Containing Links

In [ ]:
tweets = client.search_recent_tweets(
    query='from:NASA has:links', 
    expansions=['author_id'], tweet_fields=['lang'])

In [ ]:
print_tweets(tweets)

### Searching for a Hashtag

In [ ]:
tweets = client.search_recent_tweets(query='#metaverse', 
    expansions=['author_id'], tweet_fields=['lang'])

In [ ]:
print_tweets(tweets)

# 12.11 Spotting Trending Topics

In [ ]:
auth = tweepy.OAuth2BearerHandler(keys.bearer_token)

In [ ]:
api = tweepy.API(auth=auth, wait_on_rate_limit=True)

# 12.11.1 Places with Trending Topics
**Note: This part of the Twitter APIs has not been migrated from v1.1 to v2 yet and is accessible only to "Elevated" and "Academic Research" access.**

In [ ]:
available_trends = api.available_trends()

In [ ]:
len(available_trends)

In [ ]:
available_trends[0]

In [ ]:
available_trends[1]

# 12.11.2 Getting a List of Trending Topics
### Worldwide Trending Topics

In [ ]:
world_trends = api.get_place_trends(id=1)

In [ ]:
trends_list = world_trends[0]['trends']

In [ ]:
trends_list[0]

In [ ]:
trends_list = [t for t in trends_list if t['tweet_volume']]

In [ ]:
from operator import itemgetter 

In [ ]:
trends_list.sort(key=itemgetter('tweet_volume'), reverse=True) 

In [ ]:
for trend in trends_list:
    print(trend['name'])

### New York City Trending Topics

In [ ]:
nyc_trends = api.get_place_trends(id=2459115) 

In [ ]:
nyc_list = nyc_trends[0]['trends']

In [ ]:
nyc_list = [t for t in nyc_list if t['tweet_volume']]

In [ ]:
nyc_list.sort(key=itemgetter('tweet_volume'), reverse=True) 

In [ ]:
for trend in nyc_list[:5]:
    print(trend['name'])

# 12.11.3 Create a Word Cloud from Trending Topics

In [ ]:
topics = {}

In [ ]:
for trend in nyc_list:
    topics[trend['name']] = trend['tweet_volume']

In [ ]:
from wordcloud import WordCloud

In [ ]:
wordcloud = WordCloud(width=1600, height=900,
    prefer_horizontal=0.5, min_font_size=10, colormap='prism', 
    background_color='white')       

In [ ]:
wordcloud = wordcloud.fit_words(topics)

In [ ]:
wordcloud = wordcloud.to_file('TrendingTwitter.png')

**NOTE: The following code displays the image in a Jupyter Notebook**

In [ ]:
from IPython.display import Image
Image(filename='TrendingTwitter.png', width=400)

In [ ]:
##########################################################################
# (C) Copyright 2022 by Deitel & Associates, Inc. and                    #
# Pearson Education, Inc. All Rights Reserved.                           #
#                                                                        #
# DISCLAIMER: The authors and publisher of this book have used their     #
# best efforts in preparing the book. These efforts include the          #
# development, research, and testing of the theories and programs        #
# to determine their effectiveness. The authors and publisher make       #
# no warranty of any kind, expressed or implied, with regard to these    #
# programs or to the documentation contained in these books. The authors #
# and publisher shall not be liable in any event for incidental or       #
# consequential damages in connection with, or arising out of, the       #
# furnishing, performance, or use of these programs.                     #
##########################################################################
